### Step 1 — Start from consistent-resolution CMB maps

- **Space:** real/pixel space on the sphere (HEALPix maps \(T(\hat{n})\))
- **Input:** SILC temperature maps for \(N = 1, 2, 3, 4, 5\) (each already homogenized to an effective \(5'\) Gaussian beam during preprocessing), and the SMICA map.
- **Output:** same maps, ready for masking (still real space, HEALPix)


In [29]:
import numpy as np
import healpy as hp
import s2fft

def mw_alm_2_hp_alm(MW_alm, lmax):
    '''MW_alm: 2D array of shape (Lmax, 2*Lmax-1) (MW sampling, McEwen & Wiaux)
    '''
    # Initialize the 1D hp_alm array with the appropriate size
    hp_alm = np.zeros(hp.Alm.getsize(lmax), dtype=np.complex128)
        
    for l in range(lmax + 1):
        for m in range(-l, l + 1):
            index = hp.Alm.getidx(lmax, l, abs(m))
            if m < 0:
                hp_alm[index] = (-1)**m * np.conj(MW_alm[l, lmax + m])
            else:
                hp_alm[index] = MW_alm[l, lmax + m]

    return hp_alm


def hp_alm_2_mw_alm(hp_alm, L_max):
    """
    Converts spherical harmonics (alm) to a matrix representation for use in MW sampling.

    This function takes 1D Healpix spherical harmonics coefficients (alm) and converts them into a matrix form 
    that is in (MW sampling, McEwen & Wiaux) sampling. The matrix form is complex-valued 
    and indexed by multipole moment and azimuthal index.

    Parameters:
        hp_alm (numpy.ndarray): The input healpix spherical harmonics coefficients (alm).
        L_max (int): The maximum multipole moment to be represented in the output matrix.
    
    Note: # L_max = 4 | l = 0,1,2,3 , true lmax is L_max-1 = 3 | m = -3...0...(L_max-1 = 3)| number of m = 2(L_max-1)+1 = 2L_max-1

    Returns:
        MW_alm (numpy.ndarray): 2D array of shape (Lmax, 2*Lmax-1) MW spherical harmonics coefficients 
    """

    MW_alm = np.zeros((L_max, 2 * L_max - 1), dtype=np.complex128)

    for l in range(L_max):
        for m in range(-l, l + 1):
            index = hp.Alm.getidx(L_max - 1, l, abs(m))
            if m < 0:
                MW_alm[l, L_max + m - 1] = (-1) ** m * np.conj(hp_alm[index])
            else:
                MW_alm[l, L_max + m - 1] = hp_alm[index]

    return MW_alm

In [32]:
import healpy as hp

# after ILC_wav_coeff_maps_MP finishes for one realization:

component = 'CSNT'
component_name = 'cmb'
realization = "0000"  # string

# path to synthesized MW map
mw_path = f"ILC/synthesized_ILC_MW_maps/{component}_ILC_MW_Map_cilc_{component_name}_R{realization}_MP.npy"

# load MW pixel map
MW_pix = np.load(mw_path)

# if s2fft expects shape (1, ...), take first slice
if MW_pix.ndim == 3:
    MW_pix = MW_pix[0]

L = MW_pix.shape[0]
lmax = L - 1

# MW pixel → MW alm
MW_alm = s2fft.forward(MW_pix, L=L)

# MW alm → HP alm (your existing function)
hp_alm = mw_alm_2_hp_alm(MW_alm, lmax)

# choose nside for given lmax
nside = 1 << int(np.ceil(np.log2((lmax + 1) / 3.0)))

# HP alm → HP map
hp_map = hp.alm2map(hp_alm, nside=nside, lmax=lmax, verbose=False)

# save to FITS
hp.write_map(f"{component}_cILC_{component_name}_R{realization}.fits", hp_map, overwrite=True)

print(f"Saved HEALPix map to {component}_cILC_{component_name}_R{realization}.fits")


setting the output map dtype to [dtype('float64')]


Saved HEALPix map to CSNT_cILC_cmb_R0000.fits


### Step 2 — Apply the point-source mask

- **Space:** real/pixel space (multiply map by mask)  
- **Input:** map from Step 1 + point-source mask  
- **Output:** masked map $M(\hat{n})\,T(\hat{n})$ (used because Fig. 13(a) shows **point-source** masked spectra)


In [20]:
import requests
# URLs for Planck point-source masks
urls = {
    "LFI": "http://pla.esac.esa.int/pla/aio/product-action?MAP.MAP_ID=LFI_Mask_PointSrc_2048_R2.00.fits",
    "HFI": "http://pla.esac.esa.int/pla/aio/product-action?MAP.MAP_ID=HFI_Mask_PointSrc_2048_R2.00.fits"
}


# Download masks if not already present
for name, url in urls.items():
    filename = f"{name}_Mask_PointSrc_2048.fits"
    try:
        with open(filename, "wb") as f:
            f.write(requests.get(url).content)
        print(f"Downloaded {filename}")
    except Exception as e:
        print(f"Error downloading {name} mask: {e}")

Downloaded LFI_Mask_PointSrc_2048.fits
Downloaded HFI_Mask_PointSrc_2048.fits


In [31]:
       
# Load your estimated CMB map (HEALPix)
map_T = hp.read_map("CSNT_cILC_cmb_R0000.fits", verbose=False)

# Load LFI and HFI masks
mask_LFI = hp.read_map("LFI_Mask_PointSrc_2048.fits")
mask_HFI = hp.read_map("HFI_Mask_PointSrc_2048.fits")

print("map_T:", map_T.size, "nside=", hp.get_nside(map_T))
print("mask_LFI:", mask_LFI.size, "nside=", hp.get_nside(mask_LFI))
print("mask_HFI:", mask_HFI.size, "nside=", hp.get_nside(mask_HFI))

# Combine masks: intersection (product) keeps only pixels unmasked in both
mask_combined = mask_LFI * mask_HFI

# Apply mask in pixel space
masked_map = mask_combined * map_T

# Save the masked map
hp.write_map("masked_map.fits", masked_map, overwrite=True)


# visualize_MW_Pix_map(masked_map, title="Masked CMB Map", coord=["G"], unit="K", is_MW_alm=False)
# Optional: visualise
hp.mollview(masked_map, title="CMB Map with Planck Point-Source Mask")

map_T: 12288 nside= 32
mask_LFI: 50331648 nside= 2048
mask_HFI: 50331648 nside= 2048


ValueError: operands could not be broadcast together with shapes (50331648,) (12288,) 

**Step 3 — Go to harmonic space (pseudo-spectrum)**

- **Space:** harmonic space (spherical harmonics)  
- **Input:** masked map $M(\hat{n})\,T(\hat{n})$ 
- **Operation:** spherical-harmonic transform $\Rightarrow$ coefficients $\tilde{a}_{\ell m}$; compute pseudo-spectrum $\tilde{C}_{\ell}$  
- **Output:** $\tilde{C}_{\ell}$ (pseudo-$C_{\ell}$)  


In [ ]:
import numpy as np
import healpy as hp

def pseudo_cl_from_masked_map(T_map, mask, lmax=None, fsky_correct=True, return_alm=False):
    """
    Step 3 — pseudo-spectrum from a masked map, following SILC (Rogers+16).
    
    Inputs
    -------
    T_map : 1D array (len = 12*Nside^2), HEALPix CMB temperature map [K or µK]
    mask  : 1D array, same shape; 1 inside, 0 outside (can be float weights)
    lmax  : int, maximum multipole (default = 3*Nside-1)
    fsky_correct : bool, divide pseudo-Cl by f_sky (small-mask full-sky approximation)
    return_alm   : bool, also return a_lm of the masked map

    Returns
    -------
    ell        : array of multipoles (0..lmax)
    Cl_tilde   : pseudo-spectrum of the masked map, \tilde C_ell
    Cl_fullest : Cl_tilde / f_sky  (if fsky_correct), else equals Cl_tilde
    alm        : (optional) a_lm of the masked map
    """
    nside = hp.get_nside(T_map)
    if hp.get_nside(mask) != nside:
        raise ValueError("T_map and mask must have same Nside")
    if lmax is None:
        lmax = 3*nside - 1

    # Masked map M*T
    MT = (mask * T_map).astype(float)

    # Either get alm then Cl, or directly Cl; both give the pseudo-spectrum
    if return_alm:
        alm = hp.map2alm(MT, lmax=lmax, iter=0)
        Cl_tilde = hp.alm2cl(alm)
    else:
        alm = None
        Cl_tilde = hp.anafast(MT, lmax=lmax)

    # Small-mask full-sky approximation used in the paper: divide by f_sky
    # For a binary mask, f_sky = mean(mask). (Good when the mask is small.)
    if fsky_correct:
        fsky = float(mask.mean())
        Cl_fullest = Cl_tilde / fsky if fsky > 0 else Cl_tilde
    else:
        Cl_fullest = Cl_tilde

    ell = np.arange(Cl_tilde.size)
    return ell, Cl_tilde, Cl_fullest, alm
